# Average Tracts

In this script we will load a list of FIPS codes from a file, get selected demographics for those tracts and average them to produce an average tract.  

This is the final component in the goal of identifying an average tract from point data.

In [48]:
from census import Census
import pandas as pd
import math

input_file = './data/wholefoods_tracts.csv'
output_file = './data/wholefoods_data.csv'

# Read FIPS codes from file
fips_codes = []
with open(input_file,'r') as f:
    fips_codes = f.readlines()

for i in range(0, len(fips_codes)):
    fips_codes[i] = fips_codes[i].strip('\n')
    
CENSUS_API = 'bdadccb272f83b2244d1b584baef832489fd5c19'
c = Census(CENSUS_API) # Initialize census class with API key

# Generate codes for census variables of interest
var_ids = []
var_ids.extend(["B19001_0{:02d}E".format(x) for x in range(2, 18)]) # Household income over 12 months
var_ids.extend(["B19037_0{:02d}E".format(x) for x in range(1, 70)]) # Household income over 12 months by age


In [42]:
def census_data_by_fips(var_list, fips_codes):
    
    CALL_LIM = 50 # Can only request 50 records at a time
    census_dict = {}
    num_chunks = int(math.ceil(1.0 * len(var_list) / CALL_LIM))
    
    for fips in fips_codes:
        
        state_id = fips[0:2]
        county_id = fips[2:5]
        tract_id = fips[5:11]
        
        
        for chunk_num in range(0, num_chunks):
            census_data = []

            chunk_start = chunk_num * CALL_LIM
            chunk_end = (chunk_num + 1) * CALL_LIM
            if chunk_end > len(var_list): 
                chunk_end = len(var_list)

            var_sublist = var_list[chunk_start:chunk_end]

            census_data = c.acs.state_county_tract(var_sublist, state_id, county_id, tract_id)
            
            for record in census_data:
                # Eliminate original code components
                key_list = ['state', 'county', 'tract']
                for key in key_list:
                    if key in record: 
                        del record[key]

                if fips in census_dict:
                    census_dict[fips].update(record)
                else:
                    census_dict[fips] = record

    census_df = pd.DataFrame(census_dict)
    census_df = census_df.transpose()
    census_df.index.name = "fips"
    
    return census_df


In [ ]:
census_df = census_data_by_fips(var_ids, fips_codes)

In [46]:
census_df.head()

,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,...,B19037_060E,B19037_061E,B19037_062E,B19037_063E,B19037_064E,B19037_065E,B19037_066E,B19037_067E,B19037_068E,B19037_069E
fips,,,,,,,,,,,,,,,,,,,,,
06001422000,85,63,33,25,7,24,38,0,28,65,...,4,0,0,0,18,8,0,15,0,0
06001441923,92,49,128,128,189,15,62,92,56,59,...,0,37,16,0,0,36,24,19,0,0
06013339001,33,115,21,46,171,138,213,81,55,152,...,36,24,8,0,21,43,0,0,0,0
06013345108,7,41,8,0,44,22,49,74,25,185,...,13,17,14,29,31,76,28,20,62,73
06013349000,101,37,32,40,51,68,25,115,33,59,...,17,36,5,0,23,44,60,28,37,16


In [49]:
census_df.to_csv(output_file)

In [59]:
census_df = census_df.reset_index()
census_df.head()

ValueError: cannot insert level_0, already exists